# 1. Install requirements

In [ ]:
!pip install transformers

In [ ]:
!pip install --upgrade pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 30.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

#Configs

In [ ]:
data_path = "/content/train.csv"  # Update to the correct dataset path
text_column_name = "url"  # Your dataset's text column
label_column_name = "label"  # Your dataset's label column

model_name = "distilbert-base-uncased"  # Keep as is (or change if needed)
test_size = 0.2  # Keep as is (20% for testing)
num_labels = 2  # Since we have 'good' (0) and 'bad' (1), it's binary classification

# 2. Read and Prepare the Dataset

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

# Load dataset
dataset_path = "/content/train.csv"
df = pd.read_csv(dataset_path, encoding="latin1")

# Drop rows with missing labels
df = df.dropna(subset=["label"])

# Downsample Good Samples to 14k
df_good = df[df['label'] == 'good'].sample(n=14000, random_state=42)

# Keep all bad samples
df_bad = df[df['label'] == 'bad']

# Combine both datasets
df = pd.concat([df_good, df_bad]).reset_index(drop=True)

# Extract features and labels
texts = df["url"].tolist()
labels = df["label"].tolist()

# Convert labels to numerical format
label_mapping = {"bad": 1, "good": 0}
labels = [label_mapping[label] for label in labels]

print("New dataset distribution:\n", df["label"].value_counts())
print("Sample text:", texts[0])
print("Sample label:", labels[0])


New dataset distribution:
 label
bad     14307
good    14000
Name: count, dtype: int64
Sample text: familycrafts.about.com/od/thriftstorecostume/a/hccardiacarrest.htm
Sample label: 0


In [ ]:
df.head()

,url,label
0,familycrafts.about.com/od/thriftstorecostume/a...,good
1,sos.state.ia.us/business/limliabpart.html,good
2,videosurf.com/amanda-stepto-31172,good
3,biggovernment.com/capitolconfidential/2010/10/...,good
4,freecourtdockets.com/Dockets/EFM-KMH-Apsley-v-...,good


### Clean Dataset

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
class Cleaner():
  def __init__(self):
    pass
  def put_line_breaks(self,text):
    text = text.replace('</p>','</p>\n')
    return text
  def remove_html_tags(self,text):
    cleantext = BeautifulSoup(text, "lxml").text
    return cleantext
  def clean(self,text):
    text = self.put_line_breaks(text)
    text = self.remove_html_tags(text)
    return text

In [ ]:
cleaner = Cleaner()
df['text_cleaned'] = df[text_column_name].apply(cleaner.clean)

<ipython-input-9-e69a670ac7a1>:8: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a filename than HTML or XML.

If you meant to use Beautiful Soup to parse the contents of a file on disk, then something has gone wrong. You should open the file first, using code like this:

    filehandle = open(your filename)

You can then feed the open filehandle into Beautiful Soup instead of using the filename.

However, if you want to parse some data that happens to look like a filename, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  cleantext = BeautifulSoup(text, "lxml").text
<ipython-input-9-e69a670ac7a1>:8: MarkupResemblesLocatorWarning: The input pa

In [ ]:
df.head()

,url,label,text_cleaned
0,familycrafts.about.com/od/thriftstorecostume/a...,good,familycrafts.about.com/od/thriftstorecostume/a...
1,sos.state.ia.us/business/limliabpart.html,good,sos.state.ia.us/business/limliabpart.html
2,videosurf.com/amanda-stepto-31172,good,videosurf.com/amanda-stepto-31172
3,biggovernment.com/capitolconfidential/2010/10/...,good,biggovernment.com/capitolconfidential/2010/10/...
4,freecourtdockets.com/Dockets/EFM-KMH-Apsley-v-...,good,freecourtdockets.com/Dockets/EFM-KMH-Apsley-v-...


### Label Encoder

In [ ]:
from sklearn import preprocessing

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(df[label_column_name].tolist())
df['label'] = le.transform(df[label_column_name].tolist())

In [ ]:
df.head()

,url,label,text_cleaned
0,familycrafts.about.com/od/thriftstorecostume/a...,1,familycrafts.about.com/od/thriftstorecostume/a...
1,sos.state.ia.us/business/limliabpart.html,1,sos.state.ia.us/business/limliabpart.html
2,videosurf.com/amanda-stepto-31172,1,videosurf.com/amanda-stepto-31172
3,biggovernment.com/capitolconfidential/2010/10/...,1,biggovernment.com/capitolconfidential/2010/10/...
4,freecourtdockets.com/Dockets/EFM-KMH-Apsley-v-...,1,freecourtdockets.com/Dockets/EFM-KMH-Apsley-v-...


### Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train,df_test = train_test_split(df,test_size=test_size)

### Convert to Huggingface Dataset

In [ ]:
from datasets import Dataset

In [ ]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

### Tokenizer

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["text_cleaned"], truncation=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/22645 [00:00<?, ? examples/s]

In [ ]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5662 [00:00<?, ? examples/s]

# 3. Initialize Model

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    dropout=0.3,  # Sets both hidden and attention dropout
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1]),  # Explicitly convert to NumPy array
    y=df['label'].to_numpy(),
)

# 4. Train model

In [ ]:
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"  # Disable wandb completely

In [ ]:
!pip uninstall wandb -y

Found existing installation: wandb 0.19.8
Uninstalling wandb-0.19.8:
  Successfully uninstalled wandb-0.19.8


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    report_to=None,  # Disable all logging integrations (including wandb)
    learning_rate=2e-5,
    lr_scheduler_type="linear",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-33-2717f2d2d003>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.228500,0.154309,0.963794
2,0.127700,0.135134,0.968386
3,0.087100,0.147573,0.971212
4,0.056600,0.179922,0.973154
5,0.038400,0.179186,0.972978


TrainOutput(global_step=14155, training_loss=0.10765010142486367, metrics={'train_runtime': 1045.4447, 'train_samples_per_second': 108.303, 'train_steps_per_second': 13.54, 'total_flos': 1686382945450296.0, 'train_loss': 0.10765010142486367, 'epoch': 5.0})

In [ ]:
trainer.save_model('spam_model')

# 5. Evaluate Model

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
preds = trainer.predict(tokenized_train)
preds = np.argmax(preds[:3][0],axis=1)
GT = df_train['label'].tolist()
print(classification_report(GT,preds))

In [ ]:
preds = trainer.predict(tokenized_test)
preds = np.argmax(preds[:3][0],axis=1) #preds[:3][1]
GT = df_test['label'].tolist()
print(classification_report(GT,preds))